In [1]:
import os
import numpy as np
import tensorflow as tf
import pickle

In [2]:
CHECKPOINT_DIR = 'training_checkpoints'
SAVE_DIR = 'saved_model'
METADATA_DIR = 'metadata'

In [3]:
def make_mappings(alphabet):

  char2ind = {ch:ind for ind, ch in enumerate(alphabet)}
  ind2char = {ind:char for ind, char in enumerate(alphabet)}

  return alphabet, char2ind, ind2char

In [4]:
with open(os.path.join(METADATA_DIR, 'alphabet.pkl'), 'rb') as f:
    al, char2ind, ind2char = make_mappings(pickle.load(f))

In [5]:
for ch in char2ind.keys():
    assert ind2char[char2ind[ch]] == ch

for ind in ind2char.keys():
    assert char2ind[ind2char[ind]] == ind


In [6]:
RNN_UNITS = 1024
EMBEDDING_DIM = 256
VOCAB_SIZE = len(char2ind.keys())

In [7]:
tf.train.latest_checkpoint(CHECKPOINT_DIR)

'training_checkpoints/ckpt_100'

In [8]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

In [9]:
pred_model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

pred_model.load_weights(tf.train.latest_checkpoint(CHECKPOINT_DIR))

pred_model.build(tf.TensorShape([1, None]))

In [10]:
pred_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (1, None, 256)            19968     
_________________________________________________________________
gru (GRU)                    (1, None, 1024)           3938304   
_________________________________________________________________
dense (Dense)                (1, None, 78)             79950     
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


In [11]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2ind[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # We pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(ind2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [12]:
print(generate_text(pred_model, start_string=u"Proviamo "))

Proviamo a l'ore un loco vanto.
  Vedea colui che fe' di ggemo e 'l piano già scendevas veduto
si soleba colui che bene in peggior si ste della viverza.
  Poi, rompendo del tomio l'alta mira?
  El cominciò: «Sí volesse, m'è piú moviensi;
e il figlio fu da fiorentin della luna
cui men disdegno di Santa Pietase non uscise;
  ma Virgilio mi disse: «Che pu eravamo,
e piú di dubbio nella memoria mi si gira.
  Quali a veder Conte
quand'io mi trasmutai ad altra cura,
  conosceresi un divina non piú non chiede verdi sí come fiede.
  Ben se' tu manto che tosto raper quelli,
e con le ciglia ne minacciando dicea: «Quando
tu se' ogno in etterno ricco, e dentro ad esso, in su l'erba nel ciel si correda
di due radici della milla mia persona
con la sua donna e con la spirava
in pria de' suoi ben far filostro ingegno sottile?
o delli angeli, ch'ancor si mosse della pietra il vinta,
e a templ che si creda
oltre e Dio veramente piú lieve che per lo 'nferno fronda,
in nostra redenzion pur ard


In [13]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [14]:
new_model = tf.keras.models.load_model(SAVE_DIR, custom_objects={'loss': loss})

# Check its architecture
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            19968     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 78)             79950     
Total params: 4,038,222
Trainable params: 4,038,222
Non-trainable params: 0
_________________________________________________________________


In [15]:
print(generate_text(new_model, start_string=u"Proviamo "))

Proviamo a terra grita.
  Tutti cantavan: «Benedicta tu ogni pilose in su piú varso l'occhio alto,
ch'avra a sole il primo clima;
  sí vid'io ben piú di mille splendori
li morti li tiene e di cen parve me con parole scemo,
  cosí quella pacili a men per verdini
del freddo ingogni a fante e ne serra.
  Indi recherà la tua parola,
per apprezza di famigliardi si centa;
  e sí come dicea per sé stessa,
ch'uscir del primo cerchio dal becco si diparte.
  Li parversi dipirto in bene assalvarsi;
  e credenti alla tua preghieri da Marte,
or dalla Pila e Bonifazio
rimase all'un de' cigli! on sorpitra, e io tal, che noi spazio
dopo lucida, se Dio non me ne spiego.
  Prima era scendere e ciascun che certesi,
pareva ancor lo destro si stende
a piè di quello un altro coro a divorio sguardo, far noi posson con una inita cruna.
  Pier cominciò a far sua guerra,
questa, privando regno pensier ch'armove
dall'anime contenta.
  Per non soffridar: 'Maria, ora è fatto vano,
arriva'mi a l'andar
